In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-bnb-4bit",
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-7b-bnb-4bit", # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [6]:
import pandas as pd

# Lire le fichier CSV
df_csv = pd.read_csv('/content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset.csv')
print(len(df_csv))

# Supprimer les lignes en double
df_csv = df_csv.drop_duplicates()

# Enregistrer le fichier nettoyé
df_csv.to_csv('/content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset.csv', index=False)
print(len(df_csv))

9815
9815


In [7]:
from datasets import Dataset

# Convertir le DataFrame Pandas en un objet Dataset
dataset = Dataset.from_pandas(df_csv)

# Maintenant, vous pouvez utiliser la fonction map avec votre fonction de formatage
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer_model = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/9815 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [9]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.406 GB of memory reserved.


In [10]:
import torch
import xformers

print(torch.cuda.is_available())  # Cela doit retourner True
print(xformers.__version__)

True
0.0.26.post1


In [20]:
trainer_stats = trainer_model.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,815 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 159,907,840


Step,Training Loss
1,0.735200
2,1.124300
3,0.753600
4,1.018000
5,0.962200
6,1.388800
7,0.901500
8,0.971600
9,1.343300
10,0.982900


In [12]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

313.491 seconds used for training.
5.22 minutes used for training.
Peak reserved memory = 5.684 GB.
Peak reserved memory for training = 1.278 GB.
Peak reserved memory % of max memory = 38.541 %.
Peak reserved memory for training % of max memory = 8.666 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [19]:
# Charger les données depuis le fichier CSV
i = 0
data = pd.read_csv("/content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset_test.csv")

# Liste pour stocker les prédictions
predictions = []

# Parcourir chaque ligne du DataFrame
for index, row in data.iterrows():
    # Récupérer les données d'entrée
    instruction = row['instruction']
    input_text = row['input']

    # Préparer les entrées pour le modèle
    inputs = tokenizer(
        [alpaca_prompt.format(instruction, input_text, "")],
        return_tensors="pt"
    ).to("cuda")

    from transformers import TextStreamer
    # Générer la réponse
    text_streamer = TextStreamer(tokenizer)
    outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

    # Convertir les tokens en texte
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Ajouter la prédiction dans la liste
    predictions.append(output_text)

    print(" Ouput ", i, ": ", output_text)
    i += 1

# Ajouter les prédictions dans le DataFrame
data['pred_output'] = predictions

# Sauvegarder le DataFrame mis à jour en fichier CSV
data.to_csv("/content/drive/MyDrive/Classroom/MSV/predictions.csv", index=False)

print("Les prédictions ont été sauvegardées dans 'predictions.csv'.")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
### Input:
Artificial Surface 95.39


### Response:
the road is surrounded by buildings and some green trees .</s>
 Ouput  888 :  Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a short description of a remote sensing image with the following information.

### Input:
Artificial Surface 95.39


### Response:
the road is surrounded by buildings and some green trees .
<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a short description of a remote sensing image with the following information.

### Input:
Artificial Surface 98.34


### Response:
a long road is in a dense residential area .</s>
 Ouput  889 :  Below is an instruction that describes 

In [17]:
path_json = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/dataset_rsicd.json"

import json

# Charger le fichier JSON
with open(path_json, 'r') as file:
    data = json.load(file)

# Création du dataset
dataset_map = {}
for item in data["images"]:
    filename = item['filename']
    captions = [sent['raw'] for sent in item['sentences']]
    dataset_map[filename] = captions

import os
import json
import csv
import random

# Chemin du dossier contenant les fichiers de prédiction
folder_path = '/content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/test'

# Liste des fichiers texte dans le dossier
txt_files = os.listdir(folder_path)

# Créer une liste de noms de fichiers d'image correspondants
file_names = [file_name.replace('.txt', '.jpg') for file_name in txt_files]

# Chemin du fichier JSON de sortie
output_json_file_path = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset_test.json"
output_csv_file_path = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset_test.csv"

# Phrase d'instruction commune pour toutes les données
instruction = "Write a short description of a remote sensing image with the following information."

data_list = []

# Parcourir chaque fichier texte
for tmp in file_names:
    # Récupérer l'indice
    ind = file_names.index(tmp)

    # Récupérer la description de l'image à partir du fichier texte
    input_txt = ""
    with open(os.path.join(folder_path, txt_files[ind]), "r") as file:
        input_txt = file.read()

    # Récupérer une phrase de description de contexte au hasard pour l'image
    output_txt = random.choice(dataset_map[tmp])

    # Créer un dictionnaire de données
    data = {
        "input": input_txt,
        "output": output_txt,
        "instruction": instruction,
        "filename": tmp  # Ajouter le nom de fichier
    }

    # Ajouter les données à la liste
    data_list.append(data)

# Écriture des données dans le fichier JSON
with open(output_json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print("Fichier JSON créé avec succès :", output_json_file_path)

# Convertir les données JSON en CSV
csv_headers = ["input", "output", "instruction", "filename"]

with open(output_csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_headers)
    writer.writeheader()
    for data in data_list:
        writer.writerow(data)

print("Fichier CSV créé avec succès :", output_csv_file_path)


Fichier JSON créé avec succès : /content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset_test.json
Fichier CSV créé avec succès : /content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset_test.csv
